In [1]:
from __future__ import print_function, division
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.optim import Adam
import torch.nn.functional as F


import csv
from skimage import io

from PIL import Image
import pandas as pd

import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable

import matplotlib.pyplot as plt
import time
import os
import copy


import adamw.cyclic_scheduler as CyclicLRWithRestarts
import adamw.adamw as AdamW
import import_ipynb
import Caps_basics.CapsNet_Layers_multiFC as CapsNet_Layers_MFC
import Caps_basics.CapsNet_Layers as CapsNet_Layers
import Caps_basics.ResNetCaps_E as ResNetCaps_E
import Fixed_weight_loss
#torch.autograd.set_detect_anomaly(True)

CUDA, USE_CUDA, model_name, db_used = "cuda:0", True, "CapsNet_MR", "MNIST"
n_epochs, ADAM_LR, ADAM_WD= 100, 0.001,0.0
lossAN, lossML = True, True
D_simplex,U_simplex = False, False                 #<------------------False False = POLYGON
onlyML, ML_REC, onlyREC = False,True,False
batch_size = 128
alpha = 0.3

folder_results = "ALPHA_Resumed_Fixed_weight_loss_train_moreFC_"+db_used
implementation_name = str(alpha)+db_used
imageFolder = 'missClassifiedImages'
#######################
if not os.path.exists(folder_results): 
    os.mkdir(folder_results)
if not os.path.exists(os.path.join(folder_results,str(alpha)+"model_log")) and os.path.exists(folder_results): 
    os.mkdir(os.path.join(folder_results,str(alpha)+"model_log"))
implementation_folder_name = os.path.join(folder_results,implementation_name)

with open(implementation_folder_name+".txt", "w") as text_file:
    text_file.write("ADAM_LR {} ADAM_WD {} USE CUDA {} ON {} model {} dataset {} loss AN {} ML {}".format(ADAM_LR,ADAM_WD,USE_CUDA, CUDA, model_name, db_used,lossAN,lossML))
    text_file.write("loss type:onlyML {} ML_REC {} onlyREC {}".format(onlyML, ML_REC, onlyREC))
with open(implementation_folder_name+"TEST.txt", "w") as text_file:
    text_file.write("ALPHA {}, database {}\n".format(str(alpha),db_used))


def lr_decrease(optimizer, lr_decay):  
    for param_group in optimizer.param_groups:
        init_lr = param_group['lr'] 
        param_group['lr'] = init_lr*lr_decay
        
def isnan(x):
    return x != x     

def compute_loss(LossAN,LossML,loss_AN,loss_ML,alpha):
    if LossAN and not LossML:
        loss = loss_AN
    elif LossML and not LossAN:
        loss = loss_ML
    elif LossML and LossAN:
        loss = alpha*loss_AN + (1-alpha)*loss_ML
    return loss

def resume_model(name_file, model, optimizer,map_location): 
    if os.path.isfile(name_file):
        print("=> loading checkpoint '{}'".format(name_file))
        checkpoint = torch.load(name_file,map_location=map_location)
        start_epoch = checkpoint['epoch']
        model.load_state_dict(checkpoint['state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer'])
        print("=> loaded checkpoint '{}' (epoch {})"
              .format(name_file, checkpoint['epoch']))
    else:
        print("=> no checkpoint found at '{}'".format(name_file))

    return start_epoch,model,optimizer



###########################DATASET
resize_dim = (32,32)

dataset_transform = transforms.Compose([    
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

NUM_CLASSES = 10
print("MNIST")
image_datasets = {'train': datasets.MNIST('../data', train=True, download=True, transform=dataset_transform),'val': datasets.MNIST('../data', train=False, download=True, transform=dataset_transform)}
print("Initializing Datasets and Dataloaders...")
dataloaders = {'train': torch.utils.data.DataLoader(image_datasets['train'], batch_size=batch_size, shuffle=True) , 'val': torch.utils.data.DataLoader(image_datasets['val'], batch_size=batch_size, shuffle=True)}
print("Initializing Datasets and Dataloaders...")
MNIST_bo = True

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}

############################################ MODEL
print("=> using model CapsuleNET with the new loss")
USE_CUDA = True
device = torch.device(CUDA if torch.cuda.is_available() else "cpu")

model = CapsNet_Layers_MFC.CapsNet_MR(NUM_CLASSES,D_simplex,U_simplex, MNIST = MNIST_bo,smallNORB = False)

optimizer = AdamW.AdamW(model.parameters(),lr = ADAM_LR,  weight_decay=ADAM_WD)
scheduler = CyclicLRWithRestarts.CyclicLRWithRestarts(optimizer, batch_size, 60000, restart_period=5, t_mult=1.2, policy="cosine")
criterion = nn.CrossEntropyLoss().to(device)
model_path = 'model_trained/MNIST/0.3model_log/checkpoint__CapsNet_MR_40.pth.tar'

start_epoch,model,optimizer = resume_model(model_path, model, optimizer,map_location=CUDA)
for param in model.parameters():
    param.requires_grad = False
if USE_CUDA:
    model = model.to(device)#cuda()
    print('cuda')

FC_layers = CapsNet_Layers_MFC.FC_layer_single(NUM_CLASSES)    
for param in FC_layers.parameters():
    param.requires_grad = True
FC_layers.to(device)    

model_FC = nn.ModuleList()
model_FC.append(model)
model_FC.append(FC_layers)
model_FC.to(device)

criterion = nn.CrossEntropyLoss().to(device)
######NEWLOSS
criterionNew = Fixed_weight_loss.Fixed_weight_loss(device,False,False,in_feature=NUM_CLASSES,out_feature=NUM_CLASSES)
criterionNew = criterionNew.to(device)
#############


importing Jupyter notebook from /home/rita/JupyterProjects/EYE-SEA/B_CapsNet_AngleLoss/Fixed_weight_loss/Caps_basics/CapsNet_Layers_multiFC.ipynb
importing Jupyter notebook from /home/rita/JupyterProjects/EYE-SEA/B_CapsNet_AngleLoss/Fixed_weight_loss/Caps_basics/CapsNet_Layers.ipynb
importing Jupyter notebook from /home/rita/JupyterProjects/EYE-SEA/B_CapsNet_AngleLoss/Fixed_weight_loss/Caps_basics/ResNetCaps_E.ipynb
importing Jupyter notebook from Fixed_weight_loss.ipynb
MNIST
Initializing Datasets and Dataloaders...
Initializing Datasets and Dataloaders...
=> using model CapsuleNET with the new loss
=> loading checkpoint 'model_trained/MNIST/0.3model_log/checkpoint__CapsNet_MR_40.pth.tar'
=> loaded checkpoint 'model_trained/MNIST/0.3model_log/checkpoint__CapsNet_MR_40.pth.tar' (epoch 41)
cuda
margin 0  cos_m 0.0  sin_m 1.0


In [3]:
target_list = []
embeddings = []

model.train() 
scheduler.step() #<----------------------------------------------adamwr
accuracy_train,loss_train,loss_train_AN,loss_train_ML = [],[],[],[]

start = time.time()
for epoch in range(n_epochs): 
    model.train() 
    scheduler.step() #<----------------------------------------------adamwr
    print('epoch {}:{}'.format(epoch+1, n_epochs))     
    train_loss,train_loss_angle,train_loss_margin,train_accuracy = 0,0,0,0
    losted = 0
    batch_accuracy = []
    
    for batch_id, (data, target) in enumerate(dataloaders['train']):
        target =torch.eye(NUM_CLASSES).index_select(dim=0, index=target)          
        data, target = Variable(data), Variable(target)
        data, target = data.to(device), target.to(device)#.cuda()

        optimizer.zero_grad()
        _,label = torch.max(target, 1)
        
        x, recostruction,masked, output_fc = model_FC[0](data)   
        output = torch.sqrt((x**2).sum(dim=2,keepdim=True)) #<----------------v_c
        x_2D = model_FC[1](output.squeeze())

        #########NEWLOSS########
        
        target_m = []
        for i in range(len(target)):
            n_loc = (target[i,:] == 1).nonzero()
            m = torch.zeros(NUM_CLASSES,NUM_CLASSES)
            m[n_loc,n_loc] = 1
            target_m.append(m)
        del m 
        target_m = torch.stack(target_m).to(device) 
 
        if epoch == n_epochs:
            embeddings.append(x_2D)
            target_list.append(label.long())

        L_angle = criterionNew.arc_loss2D(x_2D.squeeze(),target_m,val=0)
        del target_m
#############################################only diagonal#####################################################              
        _,label = torch.max(target, 1)
#############################################only diagonal#####################################################              
        b = []
        for i in range(len(L_angle)):
            b.append(torch.diag(L_angle[i]))
        b = torch.stack(b)
        _,label = torch.max(target, 1)
        loss_AN =criterion(b,label.long())
        ################################################################################################################
# In CapsNet la loss del modello tiene di conto sia dell'output dell'encoding quanto quello del decoding
############################################## marginal loss ###################################################
        if lossML:
            if onlyML:
                loss_ML = criterionNew.margin_loss(x, target)
            elif ML_REC:
                loss_ML = criterionNew.loss(data,x,target,recostruction) 
            elif onlyREC:
                loss_ML = criterionNew.reconstruction_loss(data,recostruction)
        else:
            loss_ML = loss_AN
        loss = compute_loss(lossAN,lossML,loss_AN,loss_ML,alpha)
################################################################################################################
 
        if isnan(loss):
            #print("loss lost batch_id",batch_id)
            losted += 1
            #torch.nn.utils.clip_grad_norm_(model.parameters(), 0.25)
        else:
            
            loss.backward()
            #torch.nn.utils.clip_grad_norm_(model.parameters(), 0.25)
            optimizer.step()
            #print( batch_id)
            scheduler.batch_step() #<--------------------------------------------adamwr
            train_loss +=float(loss.data)
            train_accuracy +=float(sum(np.argmax(b.data.cpu().numpy(), 1) == np.argmax(target.data.cpu().numpy(), 1)) / float(data.size(0)))
            train_loss_angle += float(loss_AN.data)
            train_loss_margin += float(loss_ML.data)

            if batch_id % 100 == 0:
                print("BATCH_{}: train diag accuracy: {}".format(batch_id,sum(np.argmax(b.data.cpu().numpy(), 1) == 
                                       np.argmax(target.data.cpu().numpy(), 1)) / float(data.size(0))))
                print("BATCH_{}: angle loss {} margin loss {}".format(batch_id,loss_AN.data,loss_ML))

                batch_accuracy.append(float(sum(np.argmax(b.data.cpu().numpy(), 1) == 
                                       np.argmax(target.data.cpu().numpy(), 1)) / float(data.size(0))))
        del data, target
    accuracy_train.append(np.mean(batch_accuracy))
    if losted <len(dataloaders['train']):
        loss_train.append(train_loss/(len(dataloaders['train'])-losted))
        loss_train_AN.append(train_loss_angle/(len(dataloaders['train'])-losted))
        loss_train_ML.append(train_loss_margin/(len(dataloaders['train'])-losted))

    del L_angle, loss_AN, loss_ML, output_fc, masked, batch_accuracy

    
end = time.time()
print("Training time execution {}".format(end-start))
print("Loss value for training phase: {}".format(train_loss / len(dataloaders['train'])))
print("Accuracy value for training phase: {}".format(train_accuracy / len(dataloaders['train'])))

with open(implementation_folder_name+".txt", "a") as text_file:
    text_file.write("Training time execution {}\n".format(end-start))
    text_file.write("Loss value for training phase: {}\n".format(train_loss / len(dataloaders['train'])))
    text_file.write("Accuracy value for training phase: {}\n".format(train_accuracy / len(dataloaders['train'])))

epoch 1:100


/home/rita/JupyterProjects/EYE-SEA/B_CapsNet_AngleLoss/Fixed_weight_loss/Caps_basics/CapsNet_Layers_multiFC.ipynb:102: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  "        u_hat = torch.matmul(W, x)\n",
/home/rita/JupyterProjects/EYE-SEA/B_CapsNet_AngleLoss/Fixed_weight_loss/Caps_basics/CapsNet_Layers_multiFC.ipynb:141: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  "                nn.Linear(1024, out_Linear),\n",
/home/rita/.local/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/rita/.local/lib/python3.6/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


epoch 2:100
epoch 3:100
epoch 4:100
epoch 5:100
epoch 6:100
epoch 7:100
epoch 8:100
epoch 9:100
epoch 10:100
epoch 11:100
epoch 12:100
epoch 13:100
epoch 14:100
epoch 15:100
epoch 16:100
epoch 17:100
epoch 18:100
epoch 19:100
epoch 20:100
epoch 21:100
epoch 22:100
epoch 23:100
epoch 24:100
epoch 25:100
epoch 26:100
epoch 27:100
epoch 28:100
epoch 29:100
epoch 30:100
epoch 31:100
epoch 32:100
epoch 33:100
epoch 34:100
epoch 35:100
epoch 36:100
epoch 37:100
epoch 38:100
epoch 39:100
epoch 40:100
epoch 41:100
epoch 42:100
epoch 43:100
epoch 44:100
epoch 45:100
epoch 46:100
epoch 47:100
epoch 48:100
epoch 49:100
epoch 50:100
epoch 51:100
epoch 52:100
epoch 53:100
epoch 54:100
epoch 55:100
epoch 56:100
epoch 57:100
epoch 58:100
epoch 59:100
epoch 60:100
epoch 61:100
epoch 62:100
epoch 63:100
epoch 64:100
epoch 65:100
epoch 66:100
epoch 67:100
epoch 68:100
epoch 69:100
epoch 70:100
epoch 71:100
epoch 72:100
epoch 73:100
epoch 74:100
epoch 75:100
epoch 76:100
epoch 77:100
epoch 78:100
epoch 7

In [ ]:

#from mpl_toolkits.mplot3d import Axes3D 
hi = np.linspace(0, 2*np.pi, NUM_CLASSES)

rgb_cycle = np.vstack((            # Three sinusoids
    .5*(1.+np.cos(phi          )), # scaled to [0,1]
    .5*(1.+np.cos(phi+2*np.pi/3)), # 120° phase shifted.
    .5*(1.+np.cos(phi-2*np.pi/3)))).T # Shape = (60,3)
print(rgb_cycle.shape)


#fig = plt.figure()
#ax = fig.add_subplot(111, projection='3d')

for m in range(len(target_list)):
    colours = []
    target_batch = target_list[m].cpu().numpy()

    for i in target_batch:
        colours.append(rgb_cycle[i,:])
    from sklearn.decomposition import PCA 
    M = embeddings[m].squeeze().cpu().numpy()
    pca = PCA(n_components=2) 
    reduced = pca.fit_transform(M) # We need a 2 x 944 array, not 944 by 2 (all X coordinates in one list) 
    t = reduced.transpose() 
    #ax.scatter(t[0], t[1],t[2],c=target_batch) 
    plt.scatter(t[0], t[1],c=target_batch) 
plt.show()


In [ ]:

phi = np.linspace(0, 2*np.pi, NUM_CLASSES)

rgb_cycle = np.vstack((            # Three sinusoids
    .5*(1.+np.cos(phi          )), # scaled to [0,1]
    .5*(1.+np.cos(phi+2*np.pi/3)), # 120° phase shifted.
    .5*(1.+np.cos(phi-2*np.pi/3)))).T # Shape = (60,3)

fig, ax = plt.subplots(figsize=(3,3))
ax.scatter(polygon_vertex[:,0],polygon_vertex[:,1], c=rgb_cycle, s=90)
fig.show()

In [ ]:
torch.cuda.empty_cache()